In [ ]:
import pandas as pd
import numpy as np
import time
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/TAS239/rotten_tomatoes.csv")
pd.set_option("display.max_columns", None)
df.head(2)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,release_month,release_day,release_year,streaming_release_date,streaming_month,streaming_day,streaming_year,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/154,1:54,"Tim, a shy 16-year-old athlete with a natural ...",NaN,NR,"Art House & International, Drama",Yan England,Yan England,"Sophie Nélisse, Lou-Pascal Tremblay, Robert Na...",3/9/18,3.0,9.0,2018.0,3/15/18,3.0,15.0,2018.0,106.0,Breaking Glass Pictures,Rotten,57.0,7.0,Spilled,57.0,100.0,3,4,3
1,m/222_2017,2:22,Two planes almost collide after a blinding fla...,NaN,PG-13,"Drama, Mystery & Suspense",Paul Currie,"Todd Stein, Nathan Parker","Michiel Huisman, Teresa Palmer, Sam Reid, Maev...",6/30/17,6.0,30.0,2017.0,6/30/17,6.0,30.0,2017.0,99.0,Magnolia Pictures,Rotten,22.0,9.0,Spilled,38.0,599.0,1,2,7


In [ ]:
print("Total number of movies in the dataframe: " + str(len(df)))

Total number of movies in the dataframe: 17712


In [ ]:
df_movie = df[["rotten_tomatoes_link","movie_title","content_rating","genres","release_year","tomatometer_status","tomatometer_count"]]
df_movie.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count
0,m/154,1:54,NR,"Art House & International, Drama",2018.0,Rotten,7.0
1,m/222_2017,2:22,PG-13,"Drama, Mystery & Suspense",2017.0,Rotten,9.0


In [ ]:
df2 = pd.read_csv("/content/gdrive/My Drive/TAS239/RT_critic_reviews.csv")
df2.head(2)

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_status,review_date,review_content
0,m/first_man,Alex Manzano,False,Espinof,Rotten,7/11/19,"It has a good design, but zero personality. [F..."
1,m/halloween_2018,Alex Manzano,False,Espinof,Fresh,7/11/19,"It is not the ultimate horror movie, but it is..."


In [ ]:
df_review = df2[["rotten_tomatoes_link","review_status","review_content"]]
df_review.head(2)

,rotten_tomatoes_link,review_status,review_content
0,m/first_man,Rotten,"It has a good design, but zero personality. [F..."
1,m/halloween_2018,Fresh,"It is not the ultimate horror movie, but it is..."


In [ ]:
print("Total number of movies in the dataframe: " + str(len(df_review)))

Total number of movies in the dataframe: 1048575


In [ ]:
df_mrev = df_movie.join(df_review.set_index("rotten_tomatoes_link"), on = "rotten_tomatoes_link", how = "left")
df_mrev.head(2)
#df_movie information is on the left, and df_review information is on the right

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
0,m/154,1:54,NR,"Art House & International, Drama",2018.0,Rotten,7.0,Fresh,It's common for filmmakers to appear out of to...
0,m/154,1:54,NR,"Art House & International, Drama",2018.0,Rotten,7.0,Fresh,It's all quite conventional but packs a powerf...


In [ ]:
print("Total number of movies in the dataframe: " + str(len(df_mrev)))

Total number of movies in the dataframe: 1049771


#Cleaning/Restructuring my data


In [ ]:
df_mrev.isnull().sum()

,0
rotten_tomatoes_link,0
movie_title,0
content_rating,0
genres,238
release_year,21846
tomatometer_status,1417
tomatometer_count,1417
review_status,1326
review_content,62781


In [ ]:
df_mrev = df_mrev.dropna()
df_mrev.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
0,m/154,1:54,NR,"Art House & International, Drama",2018.0,Rotten,7.0,Fresh,It's common for filmmakers to appear out of to...
0,m/154,1:54,NR,"Art House & International, Drama",2018.0,Rotten,7.0,Fresh,It's all quite conventional but packs a powerf...


In [ ]:
print("Total number of movies in the dataframe after dropping all of the NaNs: " + str(len(df_mrev)))

Total number of movies in the dataframe after dropping all of the NaNs: 965158


In [ ]:
df_mrev_genres = df_mrev.assign(genres = df_mrev.genres.str.split(",")).explode("genres")
df_mrev_genres["genres"] = df_mrev_genres["genres"].str.strip()
df_mrev_genres.head(3)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
0,m/154,1:54,NR,Art House & International,2018.0,Rotten,7.0,Fresh,It's common for filmmakers to appear out of to...
0,m/154,1:54,NR,Drama,2018.0,Rotten,7.0,Fresh,It's common for filmmakers to appear out of to...
0,m/154,1:54,NR,Art House & International,2018.0,Rotten,7.0,Fresh,It's all quite conventional but packs a powerf...


In [ ]:
print("Total number of rows after exploding out the genres: " + str(len(df_mrev_genres)))

Total number of rows after exploding out the genres: 2090887


In [ ]:
genre_count = df_mrev_genres["genres"].value_counts()
genre_count

,count
genres,
Drama,545240
Comedy,311332
Action & Adventure,257565
Mystery & Suspense,207513
Science Fiction & Fantasy,153155
Romance,99722
Art House & International,94194
Horror,90629
Kids & Family,71938


In [ ]:
#When deciding what genres to include, I got rid of any genres that wouldn't contain "feel-good" movies. Opinions may differ here, but this is what I decided on.
df_mrev_genres1 = df_mrev_genres.drop(df_mrev_genres[df_mrev_genres.genres == "Action & Adventure"].index)
df_mrev_genres2 = df_mrev_genres1.drop(df_mrev_genres1[df_mrev_genres1.genres == "Mystery & Suspense"].index)
df_mrev_genres3 = df_mrev_genres2.drop(df_mrev_genres2[df_mrev_genres2.genres == "Science Fiction & Fantasy"].index)
df_mrev_genres4 = df_mrev_genres3.drop(df_mrev_genres3[df_mrev_genres3.genres == "Art House & International"].index)
df_mrev_genres5 = df_mrev_genres4.drop(df_mrev_genres4[df_mrev_genres4.genres == "Horror"].index)
df_mrev_genres6 = df_mrev_genres5.drop(df_mrev_genres5[df_mrev_genres5.genres == "Documentary"].index)
df_mrev_genres7 = df_mrev_genres6.drop(df_mrev_genres6[df_mrev_genres6.genres == "Special Interest"].index)
df_mrev_genres8 = df_mrev_genres7.drop(df_mrev_genres7[df_mrev_genres7.genres == "Musical & Performing Arts"].index)
df_mrev_genres9 = df_mrev_genres8.drop(df_mrev_genres8[df_mrev_genres8.genres == "Western"].index)
df_mrev_genres10 = df_mrev_genres9.drop(df_mrev_genres9[df_mrev_genres9.genres == "Sports & Fitness"].index)
df_mrev_genres11 = df_mrev_genres10.drop(df_mrev_genres10[df_mrev_genres10.genres == "Faith & Spirituality"].index)

df_mrev_genres11.head(10)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
3,m/1155,11:55,NR,Drama,2017.0,Fresh,5.0,Fresh,Actor Victor Almanzar makes his screenwriting ...
3,m/1155,11:55,NR,Drama,2017.0,Fresh,5.0,Fresh,It's Almanzar's controlled portrayal of a man ...
3,m/1155,11:55,NR,Drama,2017.0,Fresh,5.0,Fresh,"Sincere, sometimes brief performances - includ..."
3,m/1155,11:55,NR,Drama,2017.0,Fresh,5.0,Fresh,"Despite multiple onscreen timestamps, the film..."
3,m/1155,11:55,NR,Drama,2017.0,Fresh,5.0,Fresh,Transcends any lingering trace of gimmickry wi...
8,m/10,10,R,Comedy,1979.0,Fresh,24.0,Rotten,"And oh, the irony, that the film only deserves..."
8,m/10,10,R,Romance,1979.0,Fresh,24.0,Rotten,"And oh, the irony, that the film only deserves..."
8,m/10,10,R,Comedy,1979.0,Fresh,24.0,Fresh,This passable but not great Blake Edwards pict...
8,m/10,10,R,Romance,1979.0,Fresh,24.0,Fresh,This passable but not great Blake Edwards pict...
8,m/10,10,R,Comedy,1979.0,Fresh,24.0,Fresh,"""10"" explores sexual yearning in a drama that ..."


In [ ]:
print("Total number of movies left after filtering out the genres: " + str(len(df_mrev_genres11)))

Total number of movies left after filtering out the genres: 533845


In [ ]:
df_mrev_genres11["release_year"] = df_mrev_genres11["release_year"].astype(int)
df_mrev_genres11["tomatometer_count"] = df_mrev_genres11["tomatometer_count"].astype(int)

df_mrev_genres11.head(1)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,Actor Victor Almanzar makes his screenwriting ...


In [ ]:
df_mrev_genres11_r = df_mrev_genres11.drop(df_mrev_genres11[df_mrev_genres11.tomatometer_status == "Rotten"].index)
df_mrev_genres11_r.head(20)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,Actor Victor Almanzar makes his screenwriting ...
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,It's Almanzar's controlled portrayal of a man ...
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,"Sincere, sometimes brief performances - includ..."
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,"Despite multiple onscreen timestamps, the film..."
3,m/1155,11:55,NR,Drama,2017,Fresh,5,Fresh,Transcends any lingering trace of gimmickry wi...
8,m/10,10,R,Comedy,1979,Fresh,24,Rotten,"And oh, the irony, that the film only deserves..."
8,m/10,10,R,Romance,1979,Fresh,24,Rotten,"And oh, the irony, that the film only deserves..."
8,m/10,10,R,Comedy,1979,Fresh,24,Fresh,This passable but not great Blake Edwards pict...
8,m/10,10,R,Romance,1979,Fresh,24,Fresh,This passable but not great Blake Edwards pict...
8,m/10,10,R,Comedy,1979,Fresh,24,Fresh,"""10"" explores sexual yearning in a drama that ..."


In [ ]:
print("Total number of movies left after filtering out the 'Rotten' tomatometer status: " + str(len(df_mrev_genres11_r)))

Total number of movies left after filtering out the 'Rotten' tomatometer status: 320857


In [ ]:
display(df_mrev_genres11[(df_mrev_genres11["tomatometer_status"] == "Rotten") & (df_mrev_genres11["tomatometer_count"] > 200) & (df_mrev_genres11["genres"] == "Comedy")])

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
3751,m/demolition_2016,Demolition,R,Comedy,2016,Rotten,204,Fresh,Demolition shines due to Jake Gyllenhaal's ste...
3751,m/demolition_2016,Demolition,R,Comedy,2016,Rotten,204,Rotten,There's nothing wrong with telling the story o...
3751,m/demolition_2016,Demolition,R,Comedy,2016,Rotten,204,Rotten,All credit to Gyllenhaal though for taking a p...
3751,m/demolition_2016,Demolition,R,Comedy,2016,Rotten,204,Rotten,Plausibility clearly wasn't a prime concern here.
3751,m/demolition_2016,Demolition,R,Comedy,2016,Rotten,204,Rotten,A hymn to the fragility of an upper middle cla...
...,...,...,...,...,...,...,...,...,...
16177,m/this_is_40,This Is 40,R,Comedy,2012,Rotten,220,Fresh,Even though Apatow still throws in F bombs and...
16177,m/this_is_40,This Is 40,R,Comedy,2012,Rotten,220,Rotten,"I hate to say it, but if Judd Apatow wants to ..."
16177,m/this_is_40,This Is 40,R,Comedy,2012,Rotten,220,Rotten,"If this is what 40 is like, someone needs a re..."
16177,m/this_is_40,This Is 40,R,Comedy,2012,Rotten,220,Fresh,Whatever superficial distractions may be endem...


#**Creating my training and testing datasets**

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_mrev_genres11_r, test_size=0.2)
train.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
4515,m/everybody_wants_some,Everybody Wants Some!!,R,Comedy,2016,Certified-Fresh,240,Rotten,"The music is authentic, but to steal an SAT an..."
271,m/a_bread_factory_part_one_for_the_sake_of_gold,"A Bread Factory, Part One: For the Sake of Gold",NR,Drama,2018,Fresh,30,Fresh,"Experiences such as this, that offer such a we..."


In [ ]:
print("Total number of movies in the randomly sampled training data: " + str(len(train)))

Total number of movies in the randomly sampled training data: 256685


In [ ]:
test.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content
11898,m/solitary_man,Solitary Man,R,Comedy,2010,Certified-Fresh,98,Rotten,Douglas has always been at his best when he's ...
6343,m/how_she_move,How She Move,PG-13,Drama,2008,Fresh,79,Fresh,"The film does have great energy, and some fabu..."


In [ ]:
print("Total number of movies in the randomly sampled testing data: " + str(len(test)))

Total number of movies in the randomly sampled testing data: 64172


#**Sentiment Analysis of the Reviews**

In [ ]:
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

nltk.download("vader_lexicon")

sent = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
test["test_sentiment"] = test["review_content"].apply(lambda review_content: sent.polarity_scores(review_content))
test.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content,test_sentiment
11898,m/solitary_man,Solitary Man,R,Comedy,2010,Certified-Fresh,98,Rotten,Douglas has always been at his best when he's ...,"{'neg': 0.135, 'neu': 0.645, 'pos': 0.22, 'com..."
6343,m/how_she_move,How She Move,PG-13,Drama,2008,Fresh,79,Fresh,"The film does have great energy, and some fabu...","{'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'comp..."


In [ ]:
test["test_compound"] = test["test_sentiment"].apply(lambda sentiment_dict: sentiment_dict["compound"])
test["test_sent_type"] = ""

test.loc[test.test_compound > 0, "test_sent_type"] = "POSITIVE"
test.loc[test.test_compound == 0, "test_sent_type"] = "NEUTRAL"
test.loc[test.test_compound < 0, "test_sent_type"] = "NEGATIVE"

test.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content,test_sentiment,test_compound,test_sent_type
11898,m/solitary_man,Solitary Man,R,Comedy,2010,Certified-Fresh,98,Rotten,Douglas has always been at his best when he's ...,"{'neg': 0.135, 'neu': 0.645, 'pos': 0.22, 'com...",0.4404,POSITIVE
6343,m/how_she_move,How She Move,PG-13,Drama,2008,Fresh,79,Fresh,"The film does have great energy, and some fabu...","{'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'comp...",0.6486,POSITIVE


In [ ]:
from google.colab import files

test.to_csv("/content/gdrive/My Drive/TAS239/TEST_sentiment_" + time.strftime("%Y-%m-%d %H-%M") + ".csv")

In [ ]:
train["train_sentiment"] = train["review_content"].apply(lambda review_content: sent.polarity_scores(review_content))
train.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content,train_sentiment
4515,m/everybody_wants_some,Everybody Wants Some!!,R,Comedy,2016,Certified-Fresh,240,Rotten,"The music is authentic, but to steal an SAT an...","{'neg': 0.084, 'neu': 0.803, 'pos': 0.113, 'co..."
271,m/a_bread_factory_part_one_for_the_sake_of_gold,"A Bread Factory, Part One: For the Sake of Gold",NR,Drama,2018,Fresh,30,Fresh,"Experiences such as this, that offer such a we...","{'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'comp..."


In [ ]:
train["train_compound"] = train["train_sentiment"].apply(lambda sentiment_dict: sentiment_dict["compound"])
train["train_set_type"] = ""

train.loc[train.train_compound > 0, "train_set_type"] = "POSITIVE"
train.loc[train.train_compound == 0, "train_set_type"] = "NEUTRAL"
train.loc[train.train_compound < 0, "train_set_type"] = "NEGATIVE"

train.head(2)

,rotten_tomatoes_link,movie_title,content_rating,genres,release_year,tomatometer_status,tomatometer_count,review_status,review_content,train_sentiment,train_compound,train_set_type
4515,m/everybody_wants_some,Everybody Wants Some!!,R,Comedy,2016,Certified-Fresh,240,Rotten,"The music is authentic, but to steal an SAT an...","{'neg': 0.084, 'neu': 0.803, 'pos': 0.113, 'co...",0.1154,POSITIVE
271,m/a_bread_factory_part_one_for_the_sake_of_gold,"A Bread Factory, Part One: For the Sake of Gold",NR,Drama,2018,Fresh,30,Fresh,"Experiences such as this, that offer such a we...","{'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'comp...",0.8481,POSITIVE


In [ ]:
from google.colab import files

train.to_csv("/content/gdrive/My Drive/TAS239/TRAIN_sentiment_" + time.strftime("%Y-%m-%d %H-%M") + ".csv")